# Intro

It's time for me to do a deep dive into agents.


# What are agents?

Let's start with some definitions of agents from different sources.

::: {.callout-note}
## Agent Definition from AWS - [source](https://aws.amazon.com/what-is/ai-agents/)
An artificial intelligence (AI) agent is a software program that can interact with its environment, collect data, and use the data to perform self-determined tasks to meet predetermined goals. Humans set goals, but an AI agent independently chooses the best actions it needs to perform to achieve those goals.
:::

::: {.callout-note}
## Agent Definition from Anthropic - [source](https://www.anthropic.com/research/building-effective-agents)

**Agentic**

- Fully autonomous systems that operate independently over extended periods, using various tools to accomplish complex tasks. 
- Prescriptive implementations that follow predefined workflows. 
- At Anthropic, we categorize all these variations as **agentic** systems, but draw an important architectural distinction between **workflows** and **agents**:

**Workflows** 

- are systems where LLMs and tools are orchestrated through predefined code paths.

**Agents**

- are systems where LLMs dynamically direct their own processes and tool usage, maintaining control over how they accomplish tasks.
:::


::: {.callout-note}
## Agent Definition from Chip Huyen's Book "AI Engineering" - [source](https://learning.oreilly.com/library/view/ai-engineering/9781098166298/ch06.html#ch06_agents_1730157386572111)

An **agent** is anything that can perceive its environment and act upon that environment. This means that an **agent** is characterized by the environment it operates in and the set of actions it can perform.
:::








In [1]:
import dotenv

dotenv.load_dotenv()



True

# Resources

[OpenAI Note Book - Prerequisite to Swarm - Orchestrating Agents: Routines and Handoffs](https://github.com/openai/openai-cookbook/blob/main/examples/Orchestrating_agents.ipynb)

[Anthropic Blog - Building effective agents](https://docs.anthropic.com/en/docs/build/agents/building-effective-agentshttps://www.anthropic.com/research/building-effective-agents)

[AI Engineering Book - By Chip Huyen](https://learning.oreilly.com/library/view/ai-engineering/9781098166298/)

[Deep Learning AI Course -  Multi AI Agent Systems with crewAI](https://www.deeplearning.ai/short-courses/multi-ai-agent-systems-with-crewai/)